# Setup

In [1]:
import pandas as pd
import numpy as np

# Verificando Dados

In [2]:
train = pd.read_csv('train.csv', index_col=0)
train.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,1,...,5.0,A,A,A,A,A,A,A,B,D
2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,3,...,NaN,A,A,C,A,B,A,A,C,A
3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,F,0.0,3,...,NaN,A,A,A,A,C,A,A,B,A
4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.0,0,...,5.0,C,A,A,A,A,D,A,A,A
5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,M,0.0,2,...,NaN,A,A,A,A,A,A,A,A,A


In [3]:
train.shape

(13730, 166)

# Pré-processamento

In [4]:
y = train['IN_TREINEIRO']

In [5]:
y.shape

(13730,)

In [6]:
train = train[train['IN_TREINEIRO'] != None]

In [7]:
train.shape

(13730, 166)

In [8]:
train.dtypes[(train.dtypes == "object")].index

Index(['NU_INSCRICAO', 'NO_MUNICIPIO_RESIDENCIA', 'SG_UF_RESIDENCIA',
       'TP_SEXO', 'NO_MUNICIPIO_NASCIMENTO', 'SG_UF_NASCIMENTO',
       'NO_MUNICIPIO_ESC', 'SG_UF_ESC', 'NO_ENTIDADE_CERTIFICACAO',
       'SG_UF_ENTIDADE_CERTIFICACAO', 'NO_MUNICIPIO_PROVA', 'SG_UF_PROVA',
       'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT',
       'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC',
       'TX_RESPOSTAS_MT', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC',
       'TX_GABARITO_MT', 'Q001', 'Q002', 'Q003', 'Q004', 'Q006', 'Q007',
       'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015', 'Q016',
       'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024', 'Q025',
       'Q026', 'Q027', 'Q028', 'Q042', 'Q043', 'Q044', 'Q045', 'Q046', 'Q047',
       'Q048', 'Q049', 'Q050'],
      dtype='object')

In [9]:
new_train = train

In [10]:
new_train = new_train.drop('IN_TREINEIRO', axis=1)

In [11]:
new_train.shape

(13730, 165)

In [31]:
features = ['CO_UF_RESIDENCIA',
            'NU_IDADE',
            'TP_COR_RACA',
            'TP_NACIONALIDADE',
            'TP_ST_CONCLUSAO',
            'TP_ANO_CONCLUIU',
            'TP_ESCOLA',
            'TP_ENSINO',
            'TP_DEPENDENCIA_ADM_ESC',
            'IN_BAIXA_VISAO',
            'IN_CEGUEIRA',
            'IN_SURDEZ',
            'IN_DISLEXIA',
            'IN_DISCALCULIA',
            'IN_SABATISTA',
            'IN_GESTANTE',
            'IN_IDOSO',
            'TP_PRESENCA_CN',
            'TP_PRESENCA_CH',
            'TP_PRESENCA_LC',
            'NU_NOTA_CN',
            'NU_NOTA_CH',
            'NU_NOTA_LC',
            'TP_LINGUA',
            'TP_STATUS_REDACAO',
            'NU_NOTA_COMP1',
            'NU_NOTA_COMP2',
            'NU_NOTA_COMP3',
            'NU_NOTA_COMP4',
            'NU_NOTA_COMP5',
            'NU_NOTA_REDACAO'
           ]

In [32]:
new_train = new_train[features]

In [33]:
new_train.shape

(13730, 31)

In [34]:
new_train = new_train.fillna(0)

In [35]:
new_train.shape

(13730, 31)

In [36]:
y = np.nan_to_num(y)

In [37]:
y = y.reshape((y.shape[0], 1))

In [38]:
y.shape

(13730, 1)

# Criando modelo

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new_train, y, test_size=0.2, random_state=4)

In [40]:
from sklearn.ensemble import RandomForestClassifier
import warnings
from sklearn.model_selection import GridSearchCV

warnings.filterwarnings('ignore')

param_grid = { 
    'bootstrap': [True, False],
    'max_features': ['auto', 'log2', 'sqrt', 25, 31],
    'max_depth' : [5, 7, 10, 15, 20, 30],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
}
clf = RandomForestClassifier(random_state=4)
CV_clf = GridSearchCV(estimator=clf, param_grid=param_grid, cv=10, verbose=1, n_jobs=-1)
CV_clf.fit(X_train, y_train)

Fitting 10 folds for each of 540 candidates, totalling 5400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   13.2s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   38.5s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done 5400 out of 5400 | elapsed: 10.5min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=4,
                                  

In [45]:
y_pred = CV_clf.predict(X_test)

# Métricas

In [46]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.9970866715222141
0.989010989010989
[[2378    0]
 [   8  360]]


# Importando e tratando dataset de testes

In [47]:
test = pd.read_csv('test.csv')
test.head()

,NU_INSCRICAO,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047
0,ba0cc30ba34e7a46764c09dfc38ed83d15828897,43,RS,19,F,1,1,1,3,1,...,NaN,NaN,E,E,H,B,B,C,D,A
1,177f281c68fa032aedbd842a745da68490926cd2,15,PA,24,M,3,2,1,4,1,...,NaN,NaN,B,C,B,A,A,C,D,A
2,6cf0d8b97597d7625cdedc7bdb6c0f052286c334,29,BA,16,F,2,1,3,0,1,...,NaN,NaN,E,F,G,B,B,A,NaN,D
3,5c356d810fa57671402502cd0933e5601a2ebf1e,41,PR,17,F,1,1,2,0,2,...,40.0,480.0,E,E,E,C,B,B,C,A
4,df47c07bd881c2db3f38c6048bf77c132ad0ceb3,53,DF,19,F,1,1,1,1,1,...,80.0,720.0,E,E,E,B,B,B,D,A


In [48]:
test.shape

(4570, 43)

In [49]:
new_test = test[features]

In [50]:
new_test.shape

(4570, 31)

In [51]:
new_test = new_test.fillna(0)

In [52]:
new_test.shape

(4570, 31)

In [54]:
y_pred_final = CV_clf.predict(new_test)

In [55]:
y_pred_final = np.round(y_pred_final, 2)

In [56]:
y_pred_final

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [57]:
np.unique(y_pred_final)

array([0, 1], dtype=int64)

# Gerando arquivo csv com as predições

In [58]:
output = pd.DataFrame({'NU_INSCRICAO': test['NU_INSCRICAO'], 'IN_TREINEIRO': y_pred_final})
output.to_csv('answer.csv', index=False)